# US Superstore Sales Analysis - Data Cleaning & Preparation

In [1]:
# import necessary libraries

import pandas as pd
import numpy as np

### 1. Load the Dataset

The dataset is loaded directly from a remote source using a URL. This ensures reproducibility and avoids dependency on local file paths.


In [3]:
url = "https://raw.githubusercontent.com/KeithGalli/Pandas-Data-Science-Tasks/master/SalesAnalysis/Output/all_data.csv"
df = pd.read_csv(url)

### 2. Initial Data Inspection

I'll begin by inspecting the shape, structure, and basic contents of the dataset to understand what we are working with.


In [4]:
df.shape

(186850, 6)

In [5]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


### 3. Data Type Cleaning

Several columns require type conversion to support analysis and modeling, particularly numeric fields and the order date.


In [7]:
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], errors='coerce')
df['Price Each'] = pd.to_numeric(df['Price Each'], errors='coerce')
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

C:\Users\Grace\AppData\Local\Temp\ipykernel_3708\2880788849.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')


### 4. Handling Missing Values

Rows with missing or invalid values in critical fields are removed, as they cannot contribute meaningfully to analysis or modeling.

In [8]:
df.isna().sum()

Order ID            545
Product             545
Quantity Ordered    900
Price Each          900
Order Date          900
Purchase Address    545
dtype: int64

In [9]:
df = df.dropna()

### 5. Removing Invalid Records

I'll remove rows with non-positive quantities or prices, as these represent invalid or corrupted transactions.


In [10]:
df = df[(df['Quantity Ordered'] > 0) & (df['Price Each'] > 0)]


### 6. Feature Engineering

Now i think it's time to create additional features that will support analysis and modeling, including total sales, time-based features, and geographic indicators.


In [11]:
# Create total sales column
df['Sales'] = df['Quantity Ordered'] * df['Price Each']

# Extract date features
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Month_Name'] = df['Order Date'].dt.month_name()
df['Day'] = df['Order Date'].dt.day
df['Weekday'] = df['Order Date'].dt.day_name()
df['Hour'] = df['Order Date'].dt.hour

### 7. Extracting Location Information

Here i'll extract city and state information from the purchase address to enable geographic analysis.

In [12]:
df['City'] = df['Purchase Address'].apply(lambda x: x.split(',')[1].strip())
df['State'] = df['Purchase Address'].apply(lambda x: x.split(',')[2].split(' ')[1])

### 8. Final Dataset Review

We perform a final inspection to confirm that the dataset is clean, consistent, and ready for exploratory analysis.

In [13]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales,Year,Month,Month_Name,Day,Weekday,Hour,City,State
0,176558,USB-C Charging Cable,2.0,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",23.90,2019,4,April,19,Friday,8,Dallas,TX
2,176559,Bose SoundSport Headphones,1.0,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",99.99,2019,4,April,7,Sunday,22,Boston,MA
3,176560,Google Phone,1.0,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",600.00,2019,4,April,12,Friday,14,Los Angeles,CA
4,176560,Wired Headphones,1.0,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",11.99,2019,4,April,12,Friday,14,Los Angeles,CA
5,176561,Wired Headphones,1.0,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",11.99,2019,4,April,30,Tuesday,9,Los Angeles,CA


In [14]:
df.describe()

,Quantity Ordered,Price Each,Order Date,Sales,Year,Month,Day,Hour
count,185950.000000,185950.000000,185950,185950.000000,185950.000000,185950.000000,185950.000000,185950.000000
mean,1.124383,184.399735,2019-07-18 21:54:38.887550464,185.490917,2019.000183,7.059140,15.759532,14.413305
min,1.000000,2.990000,2019-01-01 03:07:00,2.990000,2019.000000,1.000000,1.000000,0.000000
25%,1.000000,11.950000,2019-04-16 21:05:15,11.950000,2019.000000,4.000000,8.000000,11.000000
50%,1.000000,14.950000,2019-07-17 20:40:30,14.950000,2019.000000,7.000000,16.000000,15.000000
75%,1.000000,150.000000,2019-10-26 08:14:00,150.000000,2019.000000,10.000000,23.000000,19.000000
max,9.000000,1700.000000,2020-01-01 05:13:00,3400.000000,2020.000000,12.000000,31.000000,23.000000
std,0.442793,332.731330,NaN,332.919771,0.013521,3.502996,8.782176,5.423416


### 9. Save Cleaned Dataset

The cleaned dataset is saved for use in subsequent analysis and modeling notebooks.


In [16]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df = df[df['Order Date'].dt.year == 2019]
#This was added to filter only 2019 data
df.to_csv("../data/processed/cleaned_sales.csv", index=False)